In [1]:
#data in textfile separated by ::
#so loading as textFile
lines = spark.read.text('data/mllib/als/sample_movielens_ratings.txt').rdd

In [2]:
lines.take(5)

[Row(value=u'0::2::3::1424380312'),
 Row(value=u'0::3::1::1424380312'),
 Row(value=u'0::5::2::1424380312'),
 Row(value=u'0::9::4::1424380312'),
 Row(value=u'0::11::1::1424380312')]

In [7]:
#Split based on ::
data = lines.map(lambda row:row.value.split('::'))

In [8]:
data.collect()

[[u'0', u'2', u'3', u'1424380312'],
 [u'0', u'3', u'1', u'1424380312'],
 [u'0', u'5', u'2', u'1424380312'],
 [u'0', u'9', u'4', u'1424380312'],
 [u'0', u'11', u'1', u'1424380312'],
 [u'0', u'12', u'2', u'1424380312'],
 [u'0', u'15', u'1', u'1424380312'],
 [u'0', u'17', u'1', u'1424380312'],
 [u'0', u'19', u'1', u'1424380312'],
 [u'0', u'21', u'1', u'1424380312'],
 [u'0', u'23', u'1', u'1424380312'],
 [u'0', u'26', u'3', u'1424380312'],
 [u'0', u'27', u'1', u'1424380312'],
 [u'0', u'28', u'1', u'1424380312'],
 [u'0', u'29', u'1', u'1424380312'],
 [u'0', u'30', u'1', u'1424380312'],
 [u'0', u'31', u'1', u'1424380312'],
 [u'0', u'34', u'1', u'1424380312'],
 [u'0', u'37', u'1', u'1424380312'],
 [u'0', u'41', u'2', u'1424380312'],
 [u'0', u'44', u'1', u'1424380312'],
 [u'0', u'45', u'2', u'1424380312'],
 [u'0', u'46', u'1', u'1424380312'],
 [u'0', u'47', u'1', u'1424380312'],
 [u'0', u'48', u'1', u'1424380312'],
 [u'0', u'50', u'1', u'1424380312'],
 [u'0', u'51', u'1', u'1424380312'],
 [u'0

In [9]:
from pyspark.sql import Row

rdd = data.map(lambda d: Row(userId= int(d[0]), 
                       movieId=int(d[1]), 
                       rating=int(d[2]), 
                       timestamp=long(d[3]) ))

In [10]:
rdd.take(2)

[Row(movieId=2, rating=3, timestamp=1424380312L, userId=0),
 Row(movieId=3, rating=1, timestamp=1424380312L, userId=0)]

In [11]:
df = spark.createDataFrame(rdd)

In [12]:
df.show()

+-------+------+----------+------+
|movieId|rating| timestamp|userId|
+-------+------+----------+------+
|      2|     3|1424380312|     0|
|      3|     1|1424380312|     0|
|      5|     2|1424380312|     0|
|      9|     4|1424380312|     0|
|     11|     1|1424380312|     0|
|     12|     2|1424380312|     0|
|     15|     1|1424380312|     0|
|     17|     1|1424380312|     0|
|     19|     1|1424380312|     0|
|     21|     1|1424380312|     0|
|     23|     1|1424380312|     0|
|     26|     3|1424380312|     0|
|     27|     1|1424380312|     0|
|     28|     1|1424380312|     0|
|     29|     1|1424380312|     0|
|     30|     1|1424380312|     0|
|     31|     1|1424380312|     0|
|     34|     1|1424380312|     0|
|     37|     1|1424380312|     0|
|     41|     2|1424380312|     0|
+-------+------+----------+------+
only showing top 20 rows



In [13]:
rdd.toDF().show()

+-------+------+----------+------+
|movieId|rating| timestamp|userId|
+-------+------+----------+------+
|      2|     3|1424380312|     0|
|      3|     1|1424380312|     0|
|      5|     2|1424380312|     0|
|      9|     4|1424380312|     0|
|     11|     1|1424380312|     0|
|     12|     2|1424380312|     0|
|     15|     1|1424380312|     0|
|     17|     1|1424380312|     0|
|     19|     1|1424380312|     0|
|     21|     1|1424380312|     0|
|     23|     1|1424380312|     0|
|     26|     3|1424380312|     0|
|     27|     1|1424380312|     0|
|     28|     1|1424380312|     0|
|     29|     1|1424380312|     0|
|     30|     1|1424380312|     0|
|     31|     1|1424380312|     0|
|     34|     1|1424380312|     0|
|     37|     1|1424380312|     0|
|     41|     2|1424380312|     0|
+-------+------+----------+------+
only showing top 20 rows



In [14]:
train, test = df.randomSplit([0.8,0.2])

In [15]:
from pyspark.ml.recommendation import ALS

In [16]:
als = ALS(maxIter=5, regParam=0.01, 
          userCol="userId", 
          itemCol="movieId", 
          ratingCol="rating",
          coldStartStrategy="drop"
          )

In [17]:
model = als.fit(train)

In [18]:
pred = model.transform(test)

In [ ]:
pred.take(5)

In [ ]:
from pyspark.sql.functions import *
model.recommendForAllUsers(10)[col('userId')==28].collect()

In [ ]:
df[col('userId') == 28].collect()

In [19]:
pred.show()

+-------+------+----------+------+-----------+
|movieId|rating| timestamp|userId| prediction|
+-------+------+----------+------+-----------+
|     31|     1|1424380312|    26|  0.5688443|
|     31|     4|1424380312|    12|   1.251391|
|     31|     3|1424380312|     7|  2.0280905|
|     31|     1|1424380312|     0|  1.8420422|
|     85|     1|1424380312|    15|  0.8382371|
|     85|     4|1424380312|     7|  1.7203728|
|     85|     1|1424380312|     2|  0.3185396|
|     65|     2|1424380312|     5|  3.7958956|
|     53|     1|1424380312|    12|  -5.063948|
|     78|     1|1424380312|    12|  0.7132376|
|     78|     1|1424380312|    20| 0.37253892|
|     78|     1|1424380312|    19| 0.70821506|
|     78|     1|1424380312|     4|0.059177823|
|     78|     1|1424380312|     2|  0.5212861|
|     34|     3|1424380312|    25| 0.45611653|
|     81|     3|1424380312|    26| -1.8651117|
|     81|     1|1424380312|    22|  1.5319587|
|     81|     1|1424380312|     1|  1.0473496|
|     81|    

In [20]:
from pyspark.ml.evaluation import RegressionEvaluator

In [21]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", 
                                predictionCol="prediction")

In [22]:
evaluator.evaluate(pred)

1.7800468157533558

In [23]:
model.recommendForAllUsers(10).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    28|[[92,5.156545], [...|
|    26|[[90,7.2178845], ...|
|    27|[[81,3.972104], [...|
|    12|[[35,5.0691547], ...|
|    22|[[94,6.161117], [...|
|     1|[[22,4.9475756], ...|
|    13|[[76,2.9149203], ...|
|     6|[[25,4.7109313], ...|
|    16|[[94,7.453306], [...|
|     3|[[90,5.9430723], ...|
|    20|[[55,6.04659], [4...|
|     5|[[46,5.5437684], ...|
|    19|[[94,3.9853377], ...|
|    15|[[46,5.0071797], ...|
|    17|[[55,5.2090507], ...|
|     9|[[27,5.841202], [...|
|     4|[[52,4.1182103], ...|
|     8|[[76,5.7323236], ...|
|    23|[[32,5.122225], [...|
|     7|[[25,4.7231803], ...|
+------+--------------------+
only showing top 20 rows



In [24]:
model.recommendForAllItems(10).show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     31|[[8,2.9341862], [...|
|     85|[[16,4.767917], [...|
|     65|[[20,5.025547], [...|
|     53|[[8,5.161847], [2...|
|     78|[[23,1.2297468], ...|
|     34|[[2,3.9589498], [...|
|     81|[[28,4.8980675], ...|
|     28|[[18,5.1289134], ...|
|     76|[[8,5.7323236], [...|
|     26|[[0,3.0499077], [...|
|     27|[[9,5.841202], [1...|
|     44|[[18,3.8227348], ...|
|     12|[[28,4.8653054], ...|
|     91|[[12,3.0642319], ...|
|     22|[[1,4.9475756], [...|
|     93|[[2,5.0379357], [...|
|     47|[[8,5.4026074], [...|
|      1|[[12,4.0448847], ...|
|     52|[[8,4.959272], [4...|
|     13|[[23,4.11721], [1...|
+-------+--------------------+
only showing top 20 rows

